In [1]:
%load_ext blackcellmagic

In [2]:
import torch
from torch import nn, autograd
from torch.nn import functional as F
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split
from skorch.callbacks import EarlyStopping

from nltk import casual_tokenize

from sklearn.metrics import recall_score, precision_score

import pandas as pd
import numpy as np

import csv


In [3]:
TRAIN_SPLIT = 2
MSG_LEN = 20
EMBED_DIM = 300
HIDDEN_DIM = 100
NUM_LAYERS = 2
VOCAB_SIZE = 100000
OOV_IDX = 0
EOS_IDX = 1

In [4]:
def load_wvs(embeddings_path: str, embedding_dim: int, limit=None):
    if limit is not None:
        limit = int(limit)
    with open(embeddings_path) as infile:
        if next(infile).split(" ") == embeddings_path:
            # Skip header for fasttext, don't for glove
            infile.seek(0)
        return pd.read_csv(
            infile,
            header=None,
            delim_whitespace=True,
            names=list(range(embedding_dim)),
            quoting=csv.QUOTE_NONE,
            nrows=limit,
            index_col=0,
        )


all_embeds = load_wvs("embeddings.txt", EMBED_DIM, VOCAB_SIZE)
raw_embeds = np.vstack([np.zeros(EMBED_DIM), all_embeds.values])

In [5]:
token_idxs = {tok: idx + 1 for idx, tok in enumerate(all_embeds.index.values.tolist())}

In [6]:
def vectorize(text):
    idxs = [token_idxs.get(tok, OOV_IDX) for tok in text.split(' ')[:MSG_LEN]]
    return np.array(idxs + [EOS_IDX] * (MSG_LEN - len(idxs)))

In [7]:
def make_dataset(topic_num: int):
    df = pd.read_csv("data/{}.csv".format(topic_num))
    presents = df[df["present"] == 1]
    missings = df[df["present"] == 0]
    train = pd.concat(
        [
            presents.iloc[: int(presents.shape[0] // TRAIN_SPLIT)],
            missings.iloc[: int(missings.shape[0] // TRAIN_SPLIT)],
        ],
        axis="rows",
    )
    validation = pd.concat(
        [
            presents.iloc[int(presents.shape[0] // TRAIN_SPLIT) :],
            missings.iloc[int(missings.shape[0] // TRAIN_SPLIT) :],
        ],
        axis="rows",
    )
    return train, validation


def make_x_and_y(dataset):
    x = [vectorize(text) for text in dataset['text'].values.tolist()]
    y = dataset['present'].values
    return torch.LongTensor(np.vstack(x).astype(np.int64)), \
        torch.LongTensor(y.astype(np.int64))


In [8]:
training, validation = make_dataset(36)

In [9]:
training_x, training_y = make_x_and_y(training)
validation_x, validation_y = make_x_and_y(validation)

In [10]:
training_x.shape, training_y.shape

(torch.Size([65, 20]), torch.Size([65]))

In [11]:
from skorch.dataset import Dataset

In [12]:
def prf1(predictions, true):
    p = precision_score(predictions, true)
    r = recall_score(predictions, true)
    f1 = 2 * p * r / (p + r)
    return p, r, f1

In [13]:
class Classifier(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super().__init__()
        self.word_embeddings = nn.Embedding(VOCAB_SIZE + 1, embedding_dim=EMBED_DIM)
        self.word_embeddings.weight = nn.Parameter(torch.FloatTensor(raw_embeds))
        self.word_embeddings.weight.requires_grad = False
        self.lstm = nn.GRU(
            EMBED_DIM,
            HIDDEN_DIM,
            num_layers=NUM_LAYERS,
            dropout=0.2,
            batch_first=True,
            bidirectional=True,
        )
        self.softmax = nn.Linear(HIDDEN_DIM * 4, 2)
        self.dropout = nn.Dropout2d(0.1)
        self.hidden = self.init_hidden()
    
        print(sum(param.nelement() for param in self.lstm.parameters()))
            

    def forward(self, doc):
        embeds = self.word_embeddings(doc)
        if self.training:
            embeds = self.dropout(embeds.permute(0, 2, 1)).permute(0, 2, 1)
        lstm_out, self.hidden = self.lstm(embeds)
        tag_space = self.softmax(
            torch.cat(
                [
                    torch.max(self.dropout(lstm_out), 1)[0],
                    torch.mean(self.dropout(lstm_out), 1),
                ],
                1,
            )
        )
        tag_scores = F.softmax(tag_space, dim=-1)
        return tag_scores

    def init_hidden(self):
        return (
            autograd.Variable(torch.zeros(2, 1, HIDDEN_DIM)),
            autograd.Variable(torch.zeros(2, 1, HIDDEN_DIM)),
        )


net = NeuralNetClassifier(
    Classifier,
    batch_size=32,
    max_epochs=100,
    lr=0.2,
    train_split=predefined_split(Dataset(validation_x, validation_y)),
    callbacks=[EarlyStopping(patience=5, monitor='valid_loss')],
    device='cuda',
)


In [14]:
net.fit(training_x, training_y)

422400
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.5619       0.8060        0.5481  0.0612
      2        0.7486       0.8060        0.5132  0.0376
      3        0.6660       0.8060        0.4971  0.0443
      4        0.6190       0.8060        0.4858  0.0474
      5        0.6004       0.8060        0.4770  0.0403
      6        0.5820       0.8060        0.4719  0.0610
      7        0.5688       0.8060        0.4663  0.0422
      8        0.5603       0.8060        0.4635  0.0473
      9        0.5374       0.8060        0.4599  0.0380
     10        0.5466       0.8060        0.4548  0.0469
     11        0.5262       0.8060        0.4521  0.0373
     12        0.5221       0.8060        0.4479  0.0460
     13        0.5133       0.8060        0.4440  0.0394
     14        0.5093       0.8060        0.4395  0.0470
     15        0.5003       0.8060        0.4357  0.0431
     16        0.4931   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Classifier(
    (word_embeddings): Embedding(100001, 300)
    (lstm): GRU(300, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
    (softmax): Linear(in_features=400, out_features=2, bias=True)
    (dropout): Dropout2d(p=0.1)
  ),
)

In [15]:
print('validation')
prf1(net.predict(validation_x), validation_y)

validation


(0.5384615384615384, 1.0, 0.7000000000000001)

In [16]:
print('training')
prf1(net.predict(training_x), training_y)

training


(1.0, 1.0, 1.0)

In [17]:
net.predict(torch.LongTensor(np.vstack([
    vectorize('she was not professional'),
    vectorize('she was not very professional'),
    vectorize('he was not very professional'),
    vectorize('he was very rude'),
    vectorize('she was very rude'),
    vectorize('he was very unprofessional'),
    vectorize('they were very condescending'),
    vectorize('she was very condescending'),
])))

array([0, 0, 0, 1, 1, 1, 1, 1])

In [22]:
net.batch_size = 1024

In [23]:
%%time
_ = net.predict(torch.cat([validation_x] * 1000))


CPU times: user 2.1 s, sys: 509 ms, total: 2.61 s
Wall time: 2.61 s


In [24]:
print(torch.cat([validation_x] * 1000).shape)

torch.Size([67000, 20])


In [25]:
print('gpu tokens processed per second:', 67000 * MSG_LEN / 2.6)

gpu tokens processed per second: 515384.6153846154


In [1099]:
print('cpu tokens processed per second:', 6700 * MSG_LEN / 4 / 3.35)

cpu tokens processed per second: 10000.0
